In [5]:
from PIL import Image
import cv2 as cv
import numpy as np
import time

In [6]:
path = "/home/gb/CDSAML/Images/7.jpg"

# Image Resizing

In [3]:
# Import the Images module from pillow
from PIL import Image
# Open the image by specifying the image path.
image_file = Image.open(path)
# the default
image_file.save("/home/gb/CDSAML/Images/edited.jpg", quality=50)

# Face Detection

In [4]:
path2 = "/home/gb/CDSAML/Images/edited.jpg"
face_classifier = cv.CascadeClassifier('/home/gb/CDSAML/opencv/data/haarcascades/haarcascade_frontalface_alt2.xml')
resized = cv.imread(path2)
gray = cv.cvtColor(resized, cv.COLOR_BGR2GRAY)
''' Our classifier returns the ROI of the detected face as a tuple, 
It stores the top left coordinate and the bottom right coordiantes'''
faces = face_classifier.detectMultiScale(gray, 1.0485258, 6)
'''When no faces detected, face_classifier returns and empty tuple'''
count_face = len(faces)

#if len(faces)==0:
    #print("No faces found")
'''We iterate through our faces array and draw a rectangle over each face in faces'''

for (x,y,w,h) in faces:
    cv.rectangle(resized, (x,y), (x+w,y+h), (127,0,255), 2)
    cv.imshow('Face Detection', resized)
cv.waitKey(0)
cv.destroyAllWindows()

# People and Object Detection

In [5]:

img = cv.imread(path2)
cv.imshow('window',  img)
cv.waitKey(1)

# Load names of classes and get random colors
classes = open('/home/gb/CDSAML/darknet/data/coco.names').read().strip().split('\n')
np.random.seed(42)
colors = np.random.randint(0, 255, size=(len(classes), 3), dtype='uint8')

# Give the configuration and weight files for the model and load the network.
net = cv.dnn.readNetFromDarknet('/home/gb/CDSAML/darknet/cfg/yolov3.cfg', '/home/gb/CDSAML/darknet/yolov3.weights')
net.setPreferableBackend(cv.dnn.DNN_BACKEND_OPENCV)
# net.setPreferableTarget(cv.dnn.DNN_TARGET_CPU)

# determine the output layer
ln = net.getLayerNames()
ln = [ln[i - 1] for i in net.getUnconnectedOutLayers()]

# construct a blob from the image
blob = cv.dnn.blobFromImage(img, 1/255.0, (416, 416), swapRB=True, crop=False)
r = blob[0, 0, :, :]

cv.imshow('blob', r)
text = f'Blob shape={blob.shape}'
cv.displayOverlay('blob', text)
cv.waitKey(1)

net.setInput(blob)
t0 = time.time()
outputs = net.forward(ln)
t = time.time()
#print('time=', t-t0)

#print(len(outputs))
#for out in outputs:
    #print(out.shape)

def trackbar2(x):
    confidence = x/100
    r = r0.copy()
    for output in np.vstack(outputs):
        if output[4] > confidence:
            x, y, w, h = output[:4]
            p0 = int((x-w/2)*416), int((y-h/2)*416)
            p1 = int((x+w/2)*416), int((y+h/2)*416)
            cv.rectangle(r, p0, p1, 1, 1)
    cv.imshow('blob', r)
    text = f'Bbox confidence={confidence}'
    cv.displayOverlay('blob', text)

r0 = blob[0, 0, :, :]
r = r0.copy()
cv.imshow('blob', r)
cv.createTrackbar('confidence', 'blob', 50, 101, trackbar2)
trackbar2(50)

boxes = []
confidences = []
classIDs = []
h, w = img.shape[:2]

for output in outputs:
    for detection in output:
        scores = detection[5:]
        classID = np.argmax(scores)
        confidence = scores[classID]
        if confidence > 0.5:
            box = detection[:4] * np.array([w, h, w, h])
            (centerX, centerY, width, height) = box.astype("int")
            x = int(centerX - (width / 2))
            y = int(centerY - (height / 2))
            box = [x, y, int(width), int(height)]
            boxes.append(box)
            confidences.append(float(confidence))
            classIDs.append(classID)

indices = cv.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)
count_objects = 0
count_person = 0
if len(indices) > 0:
    for i in indices.flatten():
        (x, y) = (boxes[i][0], boxes[i][1])
        (w, h) = (boxes[i][2], boxes[i][3])
        color = [int(c) for c in colors[classIDs[i]]]
        cv.rectangle(img, (x, y), (x + w, y + h), color, 2)
        text = "{}: {:.4f}".format(classes[classIDs[i]], confidences[i])
        cv.putText(img, text, (x, y - 5), cv.FONT_HERSHEY_SIMPLEX, 0.5, color, 1)
        if "person" not in text:
            count_objects += 1
        else:
            count_person += 1

cv.imshow('window', img)
cv.waitKey(0)
cv.destroyAllWindows()

In [6]:
print("Total number of objects: ",count_objects)
print("Total number of people: ",count_person)
print("Total number of face: ",count_face)

Total number of objects:  2
Total number of people:  1
Total number of face:  1
